In [1]:
from mitzu import *

sample_data_source = EventDataSource(
    connection=Connection(
        connection_type=ConnectionType.FILE,
        connection_params={"file_type": "parquet"},
        url="./samples/simple_big_data.snappy.parquet",
    ),
)
m = init_notebook_project(source=sample_data_source)

Found notebook context
Initializing project ...
Finished project initialization


In [9]:
(app_launched >> workspace_opened).config(time_group="week")